# Intro

Name:  

    Cair_Lanley

Purpose:  

    Create a Langley plot for the Cair tubes attached to 2STAR
    Assumes that it is tracking at all times, does not filter for bad tracking

Input:

    none at command line

Output:

    dict and plots 

Keywords:

    none

Dependencies:

    - numpy
    - Pyephem
    - Sun_utils
    - Cair_utils

Needed Files:

  - Cair raw calibrated comma delimited files (URC)

Modification History:

    Written: Samuel LeBlanc, On fligth from BWI to SJC, 2016-08-06
    Modified: 

# Load the required modules and prepare the paths

In [1]:
import matplotlib 
matplotlib.rc_file('C:\\Users\\sleblan2\\Research\\python_codes\\file.rc')
import matplotlib.pyplot as plt
%matplotlib notebook
import numpy as np

C:\Python27_64\WinPython-64bit-2.7.6.4\python-2.7.6.amd64\lib\site-packages\IPython\kernel\__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [2]:
import map_utils as mu
import Sun_utils as su
import Cair_utils as cu

In [3]:
fp = 'C:\\Users\\sleblan2\\Research\\4STAR\\MLO_2016\\'

# Read some files

In [4]:
f = fp+'20160702_MLO5\\CAST_001_160702_090020_URC.csv'

In [6]:
reload(cu)

<module 'Cair_utils' from 'Cair_utils.py'>

In [7]:
c = cu.read_Cair(fp+'20160702_MLO5\\CAST_001_160702_090020_URU.csv')

read first
try for second read
trying datetime parse
trying parser
toutc


In [8]:
c.keys()

['wvl',
 'Lt1020_xV',
 'DateTime',
 'Lt380_xV',
 'Lt',
 'Lt555_xV',
 'Lt510_xV',
 'DateTimeUTC',
 'Millisecond_ms',
 'Lt532_xV',
 'Lt412_xV',
 'Lt875_xV',
 'GeneralExcelTime',
 'Lt589_xV',
 'Lt625_xV',
 'Lt320_xV',
 'Lt780_xV',
 'Lt1640_xV',
 'Lt670_xV',
 'Lt443_xV',
 'Lt340_xV',
 'Lt683_xV',
 'Lt1245_xV',
 'Lt490_xV',
 'Lt710_xV']

# Run analysis and calculate the airmass and rayleigh

In [10]:
lat, lon, alt = 19.5365,-155.57615,3428.0

In [26]:
reload(su)

<module 'Sun_utils' from 'Sun_utils.py'>

## Calculate the airmass and sza

In [12]:
c = su.calc_sza_airmass(c['DateTimeUTC'],lat,lon,alt,c=c)

In [13]:
c.keys()

['wvl',
 'Lt1020_xV',
 'DateTime',
 'Lt380_xV',
 'm_aero',
 'Lt',
 'alt',
 'Lt555_xV',
 'lon',
 'Lt510_xV',
 'sza',
 'DateTimeUTC',
 'Millisecond_ms',
 'm_o3',
 'Lt532_xV',
 'Lt412_xV',
 'azi',
 'Lt875_xV',
 'GeneralExcelTime',
 'Lt589_xV',
 'Lt625_xV',
 'lat',
 'Lt320_xV',
 'Lt780_xV',
 'Lt1640_xV',
 'Lt670_xV',
 'm_ray',
 'Lt443_xV',
 'Lt340_xV',
 'Lt683_xV',
 'Lt1245_xV',
 'Lt490_xV',
 'Lt710_xV']

In [33]:
c['Lt'].shape

(19L, 151250L)

## get the rayleigh tau

In [14]:
c['wvl']

[320.0,
 340.0,
 380.0,
 412.0,
 443.0,
 490.0,
 510.0,
 532.0,
 555.0,
 589.0,
 625.0,
 670.0,
 683.0,
 710.0,
 780.0,
 875.0,
 1020.0,
 1245.0,
 1640.0]

In [34]:
c['tau_rayleigh'],c['tau_rayleigh_err'] = np.zeros_like(c['Lt']),np.zeros_like(c['Lt'])

In [35]:
for i,d in enumerate(c['DateTimeUTC']):
    c['tau_rayleigh'][:,i],c['tau_rayleigh_err'][:,i] = su.tau_rayleigh(np.array(c['wvl'])/1000.0,680.0,latitude=c['lat'],date=d)

## Calculate the 'rateaero' or Lt_aero
Which is the Lt values divided by the impact of rayleigh, and trace gases

In [45]:
c['Lt_aero'] = c['Lt']/c['sunearthf']/np.exp(-c['m_aero']*c['tau_rayleigh'])

KeyError: 'sunearthf'